In [1]:
import numpy as np
import nibabel as nib
from nilearn import datasets
import os.path as op
import os
import pandas as pd

bids_folder = '/mnt_03/ds-dnumrisk' 
subList = [f[4:6] for f in os.listdir(bids_folder) if f[0:4] == 'sub-' and len(f) == 6]

# group list
df_participants = pd.read_csv(op.join(bids_folder, 'add_tables','subjects_recruit_scan_scanned-final.csv'), header=0) #, index_col=0
group_list = df_participants.loc[:,['subject ID','group']].rename(mapper={'subject ID': 'subject'},axis=1).dropna().astype({'subject': int, 'group': int}).set_index('subject')

source_folder = op.join(bids_folder,'derivatives','correlation_matrices')
target_folder = op.join(bids_folder,'derivatives','gradients')

specification = '' # align_spec = '_align-procrustes' 

In [2]:
from utils import get_basic_mask

mask, labeling_noParcel = get_basic_mask()
N_vertices = len(np.where(mask==True)[0])


In [ ]:
# get average correlation matrix for all subs
group = 'All'

matrix_zeros = np.zeros((N_vertices, N_vertices))
av_cm = matrix_zeros.copy()

for sub in subList:
    try:
        correlation_matrix = np.load(op.join(source_folder,f'sub-{sub}_unfiltered{specification}.npy'))
        av_cm += np.arctan(correlation_matrix) # fisher-Z-transformed
        print(f'subject {sub} added')
    except:
        print(f'subject {sub} failed')

av_cm = av_cm/len(subList)
av_cm_transf = np.tan(av_cm) # sanity check: diagonal should be 1 !

np.save(op.join(source_folder,f'cm_av_ses1_fsav5_unfiltered.npy'), av_cm_transf)

In [3]:
# groups seperate

group = 0
subList_fil = [x for x in subList if group_list.loc[int(x)]['group'] == group]

av_cm = np.zeros((N_vertices, N_vertices)) # matrix with zeros
for sub in subList_fil:
    try:
        correlation_matrix = np.load(op.join(source_folder,f'sub-{sub}_unfiltered{specification}.npy'))
        av_cm += np.arctan(correlation_matrix) # fisher-Z-transformed
    except:
        print(f'subject {sub} failed')
        
av_cm = av_cm/len(subList)
av_cm_transf = np.tan(av_cm) # sanity check: diagonal should be 1 !

np.save(op.join(source_folder,f'cm_av_group-{group}.npy'), av_cm_transf)

KeyboardInterrupt: 

### now derive GMs from average CMs

In [11]:
from brainspace.gradient import GradientMaps
from brainspace.utils.parcellation import map_to_labels

sub = f'avGroup{group}'

n_components = 3 # reference gradient only has 3 components anyway... for better alignment one needs more components?! (according to Alam, 2022, L-R GM differences & cognition )

target_dir = op.join(bids_folder, 'derivatives', 'gradients', f'sub-{sub}')
if not op.exists(target_dir):
    os.makedirs(target_dir)

# load in reference gradient and apply same filter
g_ref = np.load(op.join(bids_folder,'derivatives', 'gradients','refGrad-av_task-risk_align-marg.npy')) # same labeling_noParcel as cm_unfiltered
#g_ref = g_ref[mask] # not needed here cause "unfiltered"

# now perform embedding on cleaned data + alignment
g_align_fil = GradientMaps(n_components=n_components,alignment='procrustes') # defaults: approacch = 'dm', kernel = None
g_align_fil.fit(av_cm_transf,reference=g_ref)
print(f'finished sub-{sub}: gradients generated')

gm = g_align_fil.gradients_.T # unaligned
grad = [None] * n_components
for i, g in enumerate(gm): # 
    grad[i] = map_to_labels(g, labeling_noParcel, mask=mask, fill=np.nan)
np.save(op.join(target_dir,f'sub-{sub}_gradients{specification}.npy'), grad) # save all together

gm = g_align_fil.aligned_.T # !!!! take .algined_ and not .gradients (which also exists, but those are not aligend)
grad = [None] * n_components
for i, g in enumerate(gm): # gm.gradients_.T
    grad[i] = map_to_labels(g, labeling_noParcel, mask=mask, fill=np.nan)

np.save(op.join(target_dir,f'sub-{sub}_gradients-aligned{specification}.npy'), grad) # save all together


/home/ubuntu/miniconda3/envs/numrefields/lib/python3.10/site-packages/brainspace-0.1.4-py3.10.egg/brainspace/gradient/embedding.py:70: UserWarning: Affinity is not symmetric. Making symmetric.
  warnings.warn('Affinity is not symmetric. Making symmetric.')


finished sub-avGroupAll: gradients generated
